In [111]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout, Input
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow.keras.optimizers import Adam, RMSprop
from scikeras.wrappers import KerasRegressor

In [112]:
df = pd.read_csv("화이팅_ㅋ.csv")

In [113]:
df

,주차,SCFI,코로나확진자,환율,WTI유가,나프타유가,고유황중유가
0,2013-1,1148.08,0.0,1056.49,93.56,106.6980,96.1060
1,2013-2,1232.35,0.0,1058.35,95.56,105.0360,97.7760
2,2013-3,1245.84,0.0,1075.58,95.88,103.4800,97.2140
3,2013-4,1227.84,0.0,1095.19,97.77,106.6620,97.4920
4,2013-5,1219.39,0.0,1096.00,95.72,108.1780,98.5660
...,...,...,...,...,...,...,...
584,2024-15,1757.04,39459.0,1374.30,83.14,74.9875,79.6475
585,2024-16,1769.54,31841.0,1376.43,83.85,74.3400,80.8120
586,2024-17,1940.63,34278.0,1354.63,78.11,75.0220,80.4920
587,2024-19,2305.79,33678.0,1351.76,80.06,71.8160,81.2640


In [114]:
# 날짜를 인덱스로 설정
df.set_index('주차', inplace=True)

In [115]:
# 데이터 겹치는거 행 삭제
df.iloc[563:570]

,SCFI,코로나확진자,환율,WTI유가,나프타유가,고유황중유가
주차,,,,,,
2023-52,1759.57,27732700.0,1314.16,73.81,74.1325,69.605
2023-52,1759.57,27732700.0,1314.16,73.69,74.1325,69.605
2023-52,1759.57,27732700.0,1253.50,73.81,74.1325,69.605
2023-52,1759.57,27732700.0,1253.50,73.69,74.1325,69.605
2023-52,1759.57,263390.0,1314.16,73.81,74.1325,69.605
2023-52,1759.57,263390.0,1314.16,73.69,74.1325,69.605
2023-52,1759.57,263390.0,1253.50,73.81,74.1325,69.605


In [116]:
df.drop(df.index[563:570], inplace=True)

In [117]:
df

,SCFI,코로나확진자,환율,WTI유가,나프타유가,고유황중유가
주차,,,,,,
2013-1,1148.08,0.0,1056.49,93.56,106.6980,96.1060
2013-2,1232.35,0.0,1058.35,95.56,105.0360,97.7760
2013-3,1245.84,0.0,1075.58,95.88,103.4800,97.2140
2013-4,1227.84,0.0,1095.19,97.77,106.6620,97.4920
2013-5,1219.39,0.0,1096.00,95.72,108.1780,98.5660
...,...,...,...,...,...,...
2024-15,1757.04,39459.0,1374.30,83.14,74.9875,79.6475
2024-16,1769.54,31841.0,1376.43,83.85,74.3400,80.8120
2024-17,1940.63,34278.0,1354.63,78.11,75.0220,80.4920


In [118]:
df.drop('환율', axis=1, inplace=True)

In [123]:
df

,SCFI,코로나확진자,WTI유가,나프타유가,고유황중유가
주차,,,,,
2013-1,1148.08,0.0,93.56,106.6980,96.1060
2013-2,1232.35,0.0,95.56,105.0360,97.7760
2013-3,1245.84,0.0,95.88,103.4800,97.2140
2013-4,1227.84,0.0,97.77,106.6620,97.4920
2013-5,1219.39,0.0,95.72,108.1780,98.5660
...,...,...,...,...,...
2024-15,1757.04,39459.0,83.14,74.9875,79.6475
2024-16,1769.54,31841.0,83.85,74.3400,80.8120
2024-17,1940.63,34278.0,78.11,75.0220,80.4920


In [126]:
X = df
Y = df['SCFI']

In [127]:
X

,SCFI,코로나확진자,WTI유가,나프타유가,고유황중유가
주차,,,,,
2013-1,1148.08,0.0,93.56,106.6980,96.1060
2013-2,1232.35,0.0,95.56,105.0360,97.7760
2013-3,1245.84,0.0,95.88,103.4800,97.2140
2013-4,1227.84,0.0,97.77,106.6620,97.4920
2013-5,1219.39,0.0,95.72,108.1780,98.5660
...,...,...,...,...,...
2024-15,1757.04,39459.0,83.14,74.9875,79.6475
2024-16,1769.54,31841.0,83.85,74.3400,80.8120
2024-17,1940.63,34278.0,78.11,75.0220,80.4920


In [34]:
# 데이터 정규화
scaler_X = RobustScaler()
scaled_X = scaler_X.fit_transform(X)

scaler_Y = RobustScaler()
scaled_Y = scaler_Y.fit_transform(Y.values.reshape(-1, 1))

In [13]:
# 데이터셋을 훈련, 검증 및 테스트 세트로 분할
x_train, x_temp, y_train, y_temp = train_test_split(scaled_X, scaled_Y, test_size=0.4, random_state=42)
x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# GRU 입력 형태로 데이터 변환 [samples, WINDOW steps = 1 , features]
x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_valid = x_valid.reshape((x_valid.shape[0], 1, x_valid.shape[1]))
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))

# GRU 모델 생성
model = Sequential([
    Input(shape=(1, x_train.shape[2])),
    GRU(50, return_sequences=True),
    GRU(50, return_sequences=False),
    Dropout(0.2),
    Dense(1)
])


# Model configuration
model = Sequential()
model.add(GRU(50, activation='relu', input_shape=(time_steps, X_seq.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(future_steps))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=100, validation_split=0.2)

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# 모델 훈련
history = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=100, batch_size=1, verbose=1)

# 모델 예측
train_predict = model.predict(x_train)
valid_predict = model.predict(x_valid)
test_predict = model.predict(x_test)

Epoch 1/100
348/348 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 7.8802 - mae: 1.3620 - val_loss: 0.4271 - val_mae: 0.3772
Epoch 2/100
348/348 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.5020 - mae: 0.3849 - val_loss: 0.1266 - val_mae: 0.2122
Epoch 3/100
348/348 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1818 - mae: 0.2276 - val_loss: 0.0741 - val_mae: 0.1973
Epoch 4/100
348/348 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.2955 - mae: 0.3193 - val_loss: 0.0703 - val_mae: 0.1737
Epoch 5/100
348/348 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.1545 - mae: 0.2337 - val_loss: 0.0802 - val_mae: 0.1746
Epoch 6/100
348/348 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0997 - mae: 0.1973 - val_loss: 0.0435 - val_mae: 0.1368
Epoch 7/100
348/348 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.1436 - mae: 0.2366 - val_loss: 0.0455 - val_mae: 0.1355
Epoch 8/100
348/348 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1448 - mae: 0.2117 - val_loss: 0.0725 - val_mae: 0.1652
Epoch 9/100
348/348 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/

In [16]:
# 모델 평가
mae = mean_absolute_error(y_test, test_predict)
r2 = r2_score(y_test, test_predict)

print(f'Test MAE: {mae:.3f}')
print(f'R2 Score: {r2:.3f}')

Test MAE: 0.079
R2 Score: 0.997


In [17]:
# 새로운 입력 데이터 생성 (예시 데이터,5월 31일 SCFI, 코로나 신규확진자, WTI, 나프타, 고유황중유)
new_X = np.array([[3044.77,75000, 75.57, 629.39, 600.62]])
new_X_scaled = scaler_X.transform(new_X)  # 입력 데이터를 정규화
new_X_reshaped = new_X_scaled.reshape((new_X_scaled.shape[0], 1, new_X_scaled.shape[1]))  # 입력 데이터를 GRU 입력 형태로 변환

# 예측 수행
new_Y_pred = model.predict(new_X_reshaped)

# 예측 값을 실제 값으로 변환
new_Y_pred_actual = scaler_Y.inverse_transform(new_Y_pred.reshape(-1, 1))

# 예측 결과 출력
print(f"Predicted Y (actual): {new_Y_pred_actual}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicted Y (actual): [[2809.776]]


C:\Users\user\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


In [22]:
# Get the latest value and scale it
latest_value = df[['SCFI', '코로나확진자', 'WTI유가', '나프타유가', '고유황중유가']].values[-1]
latest_value_scaled = scaler_X.transform(latest_value.reshape(1, -1))

# Prepare the input for the GRU model
current_input = latest_value_scaled.reshape((1, 1, latest_value_scaled.shape[1]))

# Assuming the model is capable of predicting multiple future steps in one go
future_steps = 1
predicted = model.predict(current_input)

# Check if the model can output the required number of future steps directly
if predicted.shape[1] < future_steps:
    print(f"Model does not support predicting {future_steps} steps in one go.")
else:
    future_predictions = predicted[0, :future_steps]

    # Inverse transform the predictions
    future_predictions = scaler_Y.inverse_transform(future_predictions.reshape(-1, 1))

    # Print the predictions
    print(f"Future Predictions: {future_predictions}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Model does not support predicting 4 steps in one go.


C:\Users\user\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


In [93]:
# Create sequences for multi-step prediction
def create_sequences(X, y, time_steps=1, future_steps=4):
    Xs, ys = [], []
    for i in range(len(X) - time_steps - future_steps + 1):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[(i + time_steps):(i + time_steps + future_steps)])
    return np.array(Xs), np.array(ys)

time_steps = 1
future_steps = 1

X_seq, y_seq = create_sequences(scaled_X, scaled_Y, time_steps, future_steps)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# Build the GRU model
model = Sequential()
model.add(GRU(50, activation='relu', input_shape=(time_steps, X_seq.shape[2])))
model.add(Dense(future_steps))  # Output layer for multiple future steps

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=50, validation_split=0.2)

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model

# Get the latest value and scale it
latest_value = df[['SCFI', '코로나확진자', 'WTI유가', '나프타유가', '고유황중유가']].values[-time_steps:]
latest_value_scaled = scaler_X.transform(latest_value)

# Prepare the input for the GRU model
current_input = latest_value_scaled.reshape((1, time_steps, latest_value_scaled.shape[1]))

# Predict multiple future steps
predicted = model.predict(current_input)

# Inverse transform the predictions
future_predictions = scaler_Y.inverse_transform(predicted.reshape(-1, 1))

# Print the predictions
print(f"Future Predictions: {future_predictions}")

Epoch 1/50


C:\Users\user\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - loss: 11.9288 - val_loss: 13.3660
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 10.6639 - val_loss: 11.0852
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 9.3663 - val_loss: 7.3260
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 4.9393 - val_loss: 4.4125
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 3.1447 - val_loss: 1.7028
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.6392 - val_loss: 1.3073
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.0121 - val_loss: 1.0758
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.8148 - val_loss: 0.9431
Epoch 9/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.7438 - val_loss: 0.7576
Epoch 10/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.6661 - val_loss: 0.6947
Epoch 11/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.5101 - val_loss: 0.5871
Epoch 12/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.4476 - val_loss

C:\Users\user\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 359ms/step
Future Predictions: [[1886.7096]
 [2201.0598]
 [2310.5515]
 [1940.7665]
 [2365.9124]
 [2330.4773]
 [2215.8489]
 [2216.1416]]


In [36]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import mean_squared_error

# Get the latest value and scale it
latest_value = df[['SCFI', '코로나확진자', 'WTI유가', '나프타유가', '고유황중유가']].values[-time_steps:]
latest_value_scaled = scaler_X.transform(latest_value)

# Prepare the input for the GRU model
current_input = latest_value_scaled.reshape((1, time_steps, latest_value_scaled.shape[1]))

# Predict multiple future steps
predicted = model.predict(current_input)

# Inverse transform the predictions
future_predictions = scaler_Y.inverse_transform(predicted.reshape(-1, 1))

# Print the predictions
print(f"Future Predictions: {future_predictions}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Future Predictions: [[2153.2776]
 [2207.6052]
 [2262.079 ]
 [2495.9204]]


C:\Users\user\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


In [94]:
# 마지막 14번째 행부터 10개의 행을 타임 스텝으로 사용
start_index = -16
end_index = start_index + time_steps
latest_value = df[['SCFI', '코로나확진자', 'WTI유가', '나프타유가', '고유황중유가']].values[start_index:end_index]
latest_value_scaled = scaler_X.transform(latest_value)
current_input = latest_value_scaled.reshape((1, time_steps, latest_value_scaled.shape[1]))

# 미래 4개 값 예측
predicted_future = model.predict(current_input)
predicted_future_inv = scaler_Y.inverse_transform(predicted_future.reshape(-1, 1))
print(predicted_future_inv)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[1742.8596]
 [1787.5312]
 [1741.7887]
 [1719.3794]
 [1893.1537]
 [1970.565 ]
 [1884.3944]
 [2090.924 ]]


C:\Users\user\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


In [95]:
# 모델 평가
mae = mean_absolute_error(df['SCFI'][-8:], predicted_future_inv)
r2 = r2_score(df['SCFI'][-8:], predicted_future_inv)

print(f'Test MAE: {mae:.3f}')
print(f'R2 Score: {r2:.3f}')

Test MAE: 139.115
R2 Score: 0.418


In [135]:
scaler_X = RobustScaler()
scaled_X = scaler_X.fit_transform(X)

scaler_Y = RobustScaler()
scaled_Y = scaler_Y.fit_transform(Y.values.reshape(-1, 1))

# Sequence creation function
def create_sequences(X, Y, time_steps=1, future_steps=4):
    Xs, ys = [], []
    for i in range(len(X) - time_steps - future_steps + 1):
        Xs.append(X[i:(i + time_steps)])
        ys.append(Y[(i + time_steps):(i + time_steps + future_steps)])
    return np.array(Xs), np.array(ys)

time_steps = 8
future_steps = 8

X_seq, y_seq = create_sequences(scaled_X, scaled_Y, time_steps, future_steps)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# Model configuration
model = Sequential()
model.add(GRU(50, activation='relu', input_shape=(time_steps, X_seq.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(future_steps))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=100, validation_split=0.2)

# Use the last 16 rows of the dataframe for the prediction input
start_index = -16
end_index = start_index + time_steps
latest_value = df[['SCFI', '코로나확진자', 'WTI유가', '나프타유가', '고유황중유가']].values[start_index:end_index]
latest_value_scaled = scaler_X.transform(latest_value)
current_input = latest_value_scaled.reshape((1, time_steps, latest_value_scaled.shape[1]))

# Predict the future 8 values
predicted_future = model.predict(current_input)
predicted_future_inv = scaler_Y.inverse_transform(predicted_future.reshape(-1, 1))

# Evaluate on the entire test set
test_predict = model.predict(X_test)
test_predict_inv = scaler_Y.inverse_transform(test_predict.reshape(-1, future_steps))
y_test_inv = scaler_Y.inverse_transform(y_test.reshape(-1, future_steps))

mae_test = mean_absolute_error(y_test_inv, test_predict_inv)
r2_test = r2_score(y_test_inv, test_predict_inv)

print(f'Test MAE: {mae_test:.3f}')
print(f'Test R2 Score: {r2_test:.3f}')

Epoch 1/100


C:\Users\user\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 91ms/step - loss: 13.2667 - val_loss: 12.4855
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 9.9046 - val_loss: 9.2503
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 6.2887 - val_loss: 4.4417
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 3.0107 - val_loss: 2.0464
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.9986 - val_loss: 0.9267
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.4718 - val_loss: 0.8170
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 1.3138 - val_loss: 0.7323
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.0965 - val_loss: 0.7087
Epoch 9/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.1373 - val_loss: 0.7206
Epoch 10/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.3520 - val_loss: 0.6285
Epoch 11/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.9501 - val_loss: 0.6180
Epoch 12/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 1

C:\Users\user\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 528ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step  
Test MAE: 106.427
Test R2 Score: 0.977
